In [180]:
kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2016)
folds = kf.split(range(len(y)))
S_train_i = np.zeros(len(y))
print len(y)
for j, (train_idx, test_idx) in enumerate(folds):
    print (train_idx, test_idx)

30396
(array([    1,     2,     3, ..., 30393, 30394, 30395]), array([    0,     5,    10, ..., 30385, 30388, 30392]))
(array([    0,     1,     2, ..., 30391, 30392, 30395]), array([    9,    16,    18, ..., 30382, 30393, 30394]))
(array([    0,     1,     2, ..., 30393, 30394, 30395]), array([    4,     6,     7, ..., 30377, 30384, 30389]))
(array([    0,     4,     5, ..., 30393, 30394, 30395]), array([    1,     2,     3, ..., 30378, 30390, 30391]))
(array([    0,     1,     2, ..., 30392, 30393, 30394]), array([   15,    20,    26, ..., 30386, 30387, 30395]))


In [155]:
ID = np.array([1,2,3])

In [154]:
df = pd.DataFrame({'id':[1,2], 'y':[4,5]})

In [157]:
df2 = pd.DataFrame(ID, columns=['id'])

In [158]:
df2.merge(df, on='id', how='left')

,id,y
0,1,4.0
1,2,5.0
2,3,NaN


In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import model_selection, preprocessing
from sklearn import ensemble   
from sklearn import datasets   
from sklearn.utils import shuffle   
import xgboost as xgb
color = sns.color_palette()

%matplotlib inline

pd.options.mode.chained_assignment = None  # default='warn'
pd.set_option('display.max_columns', 500)

In [2]:
class XgbModel(object):
    def __init__(self, params, num_boost_rounds):
        self.params = params
        self.num_boost_rounds = num_boost_rounds
        
    def copy(self):
        return XgbModel(self.params, self.num_boost_rounds)
    
    def fit(self, X, y):
        xgtrain = xgb.DMatrix(X, y)
        self.model = xgb.train(self.params, xgtrain, self.num_boost_rounds)
        
    def predict(self, X):
        xgtest = xgb.DMatrix(X)
        return self.model.predict(xgtest)

In [3]:
from stacking import Stacking

In [275]:
import stacking

In [276]:
reload(stacking)

<module 'stacking' from 'stacking.py'>

In [4]:
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

In [5]:
def logrmse(predictions, targets):
    return rmse(np.log(predictions), np.log(targets))

# bruno

In [6]:
import bruno

In [34]:
reload(bruno)

<module 'bruno' from 'bruno.py'>

In [45]:
xgb_params = {
    'eta': 0.05,
    'max_depth': 5,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 0
}
num_boost_rounds = 420  # From Bruno's original CV, I think
bruno_xgb = XgbModel(xgb_params, num_boost_rounds)

# gunja

In [9]:
import gunja

In [10]:
reload(gunja)

<module 'gunja' from 'gunja.pyc'>

In [44]:
xgb_params = {
    'eta': 0.05,
    'max_depth': 6,
    'subsample': 0.6,
    'colsample_bytree': 1,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 0
}
num_boost_rounds = 422
gunja_xgb = XgbModel(xgb_params, num_boost_rounds)

# louis

In [12]:
import louis

In [13]:
reload(louis)

<module 'louis' from 'louis.pyc'>

In [43]:
xgb_params = {
    'eta': 0.05,
    'max_depth': 5,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 0
}
num_boost_rounds = 385  # This was the CV output, as earlier version shows
louis_xgb = XgbModel(xgb_params, num_boost_rounds)

# stacking

In [32]:
id_train = pd.read_csv('../input/train.csv')['id'].values

In [23]:
class DataResolver(object):
    def __init__(self):
        self.__time = -1
    
    def next(self):
        self.__time = self.__time + 1
        if (self.__time >= 2):
            return louis.resolver()
        if (self.__time >= 1):
            return gunja.resolver()
        return bruno.resolver()

In [47]:
base_models = []
base_models.append(bruno_xgb)
base_models.append(gunja_xgb)
base_models.append(louis_xgb)
stacking = Stacking(5, base_models, DataResolver(), logrmse)

In [ ]:
s_train, s_test = stacking.fit(id_train, 7662)#38132 30471

(38132, 390)
(38132L, 394L)
(38132L, 394L)
model 0
train: 0.420613501748 val: 0.477737831655
train: 0.425007006396 val: 0.459305298253
train: 0.42039672604 val: 0.480549373028


In [46]:
s_train

array([[ 0.5,  0.5,  0.5],
       [ 0.5,  0.5,  0.5],
       [ 0.5,  0.5,  0.5],
       ..., 
       [ 0.5,  0.5,  0.5],
       [ 0.5,  0.5,  0.5],
       [ 0.5,  0.5,  0.5]])

In [288]:
s_train

array([[  4285453.5 ,   4452850.  ,   4986183.5 ],
       [  4463656.5 ,   3639487.  ,   5244543.5 ],
       [  4424056.5 ,   4173486.25,   4886411.  ],
       ..., 
       [  5769501.  ,   5717440.  ,   5201236.5 ],
       [ 10980000.  ,  10370804.  ,  10273804.  ],
       [  6817400.  ,   7259843.5 ,   6272357.  ]])

In [290]:
y

array([  4982360.33215714,   5110113.16118681,   4854607.50312747, ...,
         7347390.786     ,  14229000.        ,   5902400.        ])

In [289]:
s_test

array([[ 5652413.16666667,  5861349.        ,  5300566.5       ],
       [ 8181883.66666667,  8146179.66666667,  8037454.33333333],
       [ 5426677.5       ,  5260439.16666667,  5407290.83333333],
       ..., 
       [ 3811904.91666667,  3670960.08333333,  5035616.83333333],
       [ 3996235.41666667,  4012868.83333333,  5233059.5       ],
       [ 7929215.33333333,  8412150.66666667,  8697921.33333333]])

In [291]:
xgb_params = {
    'eta': 0.05,
    'max_depth': 6,
    'subsample': 0.6,
    'colsample_bytree': 1,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 1
}

dtrain = xgb.DMatrix(s_train, y)
dtest = xgb.DMatrix(s_test)


num_boost_rounds = 100
model = xgb.train(dict(xgb_params, silent=0), dtrain, num_boost_round=num_boost_rounds)


y_predict = model.predict(dtest)

NameError: name 'id_test' is not defined

In [292]:

gunja_output = pd.DataFrame({'id': pd.read_csv('../input/test.csv').id, 'price_doc': y_predict}) 

In [293]:
gunja_output.to_csv('../stacking.csv', index=False)

In [286]:
id, X, y, T = gunja.resolver()

In [287]:
y.shape

(30396L,)

In [187]:
id, X, y, T = louis.resolver()

In [134]:
X, y, T = bruno.Resolver().get()

(38132, 391)
(38132L, 395L)
(38132L, 395L)


In [101]:
louis_xgb.fit(X, y)

In [102]:
y_pred = louis_xgb.predict(X)

In [103]:
logrmse(y, y_pred)

2.9603244826493116